In [237]:
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline
import fuzzy as fy
from slice import Slice
from collections import namedtuple
from hyphen import Hyphenator
from hyphen.dictools import *
from sklearn.cluster import KMeans
from kmodes.kmodes import KModes
import numpy as np
from numpy.linalg import svd
from sklearn.decomposition import NMF
from numpy import array, matrix, linalg

In [141]:
df = pd.read_csv('StateNames.csv')

In [5]:
#This definition of popularity (75th percentile of the entire dataset)
#could be more refined--could be stated as the 75th percentile
#relative to that year's statistics
popular = df[(df.Count >= 34)]

In [6]:
pops = popular.sample(100000)

In [14]:
def clean_data(df):
    soundex = fy.Soundex(4)
    
    df['Soundex'] = df.Name.apply(soundex)
#     df['nyiis'] = df.Name.apply(fy.nysiis)
    df['new_name'] = df.Name.str.lower()
    
    df['sound_strip'] = df.Soundex.map(lambda x: x.strip('0'))
    df['first_syl'] = df.sound_strip.str[0]
    df['last_syl'] = df.sound_strip.str[-1]

    #Read in biblical names, create biblical variable
#     old_test = pd.read_table('old-testament.txt')
#     old_test.columns = ['biblical']
#     names_list = [name.lower() for name in old_test.biblical]
#     cond_list = '|'.join(names_list)
#     df['biblical'] = df.new_name.str.contains(cond_list)
    
    
    df = df.drop(['Name', 'sound_strip', 'Soundex', 'Id'], axis=1)
    
    
    return df

In [15]:
test = clean_data(pops)

In [16]:
test.head()

,Id,Year,Gender,State,Count,nyiis,new_name,first_syl,last_syl
4816735,4816736,1967,F,TX,111,RASALAND,rosalinda,R,5
4622567,4622568,1910,F,TN,83,FRANC,frances,F,2
1111142,1111143,1982,M,FL,40,HAD,howard,H,3
5330329,5330330,1934,F,WA,100,MARLAN,marlene,M,5
1695765,1695766,1985,M,IL,97,JANATAN,johnathan,J,5


In [19]:
# en_US = Hyphenator('en_US')

In [21]:
test = test.drop('nyiis', axis=1)

In [24]:
lat_lon = pd.read_csv('state_latlon.csv')

In [29]:
test = pd.merge(test,lat_lon, how='left', left_on = 'State', right_on='state')

In [32]:
test = test.drop('state', axis=1)

In [35]:
replicate = test[:10]

In [61]:
# for row in replicate.iterrows():

In [52]:
test = test.drop('Id', axis=1)

In [63]:
test = test.drop('Count', axis=1)

In [64]:
km = KModes()
km.fit(test)

KModes(init='Cao', max_iter=100, n_clusters=8, n_init=1, verbose=0)

In [69]:
top_centroids = km.cluster_centroids_.argsort()[:,-1:-11:-1]

In [138]:
# for num, centroid in enumerate(top_centroids):
#     print "%d: %s" % (num, ", ".join(test.columns[i] for i in centroid))

In [110]:
centroids = km.cluster_centroids_

In [103]:
# tuples = zip(km.enc_map_[0].keys(), km.enc_map_[0].values())
encoded_vals = km.enc_map_

In [121]:
centroids

array([[99, 0, 4, 1569, 9, 4, 14, 4],
       [97, 1, 43, 1392, 0, 1, 2, 16],
       [104, 1, 34, 3054, 12, 4, 35, 43],
       [98, 0, 14, 2146, 12, 1, 28, 26],
       [91, 0, 38, 206, 0, 4, 30, 39],
       [94, 0, 35, 2109, 12, 4, 29, 33],
       [98, 0, 9, 531, 2, 4, 1, 34],
       [92, 0, 22, 2686, 18, 4, 38, 31]], dtype=object)

In [111]:
def inverse(encoded_list):
    inv_list = []

    for i in encoded_list:
        inv_list.append({v: k for k, v in i.items()})
    return inv_list


In [116]:
decoded_vals = inverse(encoded_vals)

In [134]:
def return_attributes(centroids, decoded_vals):
    centroid_attributes = []
    for i, row in enumerate(centroids.T):
        row_attributes = []
        for val in row:
            row_attributes.append(decoded_vals[i][val])
        centroid_attributes.append(row_attributes)
    return np.array(centroid_attributes).T

In [135]:
return_attributes(centroids, decoded_vals)

array([['2009', 'F', 'CA', 'joseph', 'J', '5', '36.17', '-119.7462'],
       ['2007', 'M', 'TX', 'james', 'A', '2', '31.106', '-97.6475'],
       ['2014', 'M', 'NY', 'william', 'M', '5', '42.1497', '-74.9384'],
       ['2008', 'F', 'IL', 'mary', 'M', '2', '40.3363', '-89.0022'],
       ['2001', 'F', 'PA', 'anna', 'A', '5', '40.5773', '-77.264'],
       ['2004', 'F', 'OH', 'marilyn', 'M', '5', '40.3736', '-82.7755'],
       ['2008', 'F', 'FL', 'catherine', 'C', '5', '27.8333', '-81.717'],
       ['2002', 'F', 'MI', 'sarah', 'S', '5', '43.3504', '-84.5603']], 
      dtype='|S21')

In [137]:
test.groupby(['Year','new_name'])  

In [ ]:
def pivot_data(df):

In [143]:
df['new_name'] = df.Name.str.lower()

In [147]:
df = df.set_index(['Year','State'])

In [150]:
#df.drop(['Year', 'State'])
df['name_gender'] = zip(df.new_name, df.Gender)

In [151]:
df = df.drop(['Year', 'State', 'new_name'])

Id      Name Gender  Count  new_name    name_gender
Year State                                                     
1910 AK      1      Mary      F     14      mary      (mary, F)
     AK      2     Annie      F     12     annie     (annie, F)
     AK      3      Anna      F     10      anna      (anna, F)
     AK      4  Margaret      F      8  margaret  (margaret, F)
     AK      5     Helen      F      7     helen     (helen, F)

In [153]:
sample = df.sample(100000)

In [191]:
date_state_series = sample.groupby(sample.index).sum()['Count']

In [173]:
total_dict = date_state_series.to_dict()

In [175]:
sample['tuple_index'] = sample.index.values

In [205]:
sample = sample.join(date_state_series, how='left', on='tuple_index', lsuffix='name_count')

In [206]:
sample['count_norm'] = sample.Countname_count / sample.Count

,,Id,Name,Gender,Countname_count,new_name,name_gender,tuple_index,Count,count_norm
Year,State,,,,,,,,,
2000,AR,195920,Caroline,F,44,caroline,"(caroline, F)","(2000, AR)",395,0.111392
1954,NJ,3430033,Gabriel,M,6,gabriel,"(gabriel, M)","(1954, NJ)",1784,0.003363
1973,MI,2598484,Brion,M,6,brion,"(brion, M)","(1973, MI)",1158,0.005181
2001,TX,4890503,Lexia,F,8,lexia,"(lexia, F)","(2001, TX)",4192,0.001908
2003,WA,5427310,Siddharth,M,5,siddharth,"(siddharth, M)","(2003, WA)",733,0.006821


In [208]:
pivot = pd.pivot_table(sample, values='count_norm',index=sample.index, columns=sample.name_gender)

Index([(1910, u'AL'), (1910, u'AR'), (1910, u'AZ'), (1910, u'CA'),
       (1910, u'CO'), (1910, u'CT'), (1910, u'DC'), (1910, u'FL'),
       (1910, u'GA'), (1910, u'HI'),
       ...
       (2014, u'SD'), (2014, u'TN'), (2014, u'TX'), (2014, u'UT'),
       (2014, u'VA'), (2014, u'VT'), (2014, u'WA'), (2014, u'WI'),
       (2014, u'WV'), (2014, u'WY')],
      dtype='object', length=5317)

In [210]:
pivot.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5317 entries, (1910, AL) to (2014, WY)
Columns: 10991 entries, (aaden, M) to (zyriah, F)
dtypes: float64(10991)
memory usage: 445.9+ MB


In [212]:
#Pivoted data with (names, sex) as features and (year, state) as the index
pivot = pivot.fillna(0)

In [214]:
U, sigma, VT = svd(pivot)

In [215]:
U.shape

(5317, 5317)

In [216]:
VT.shape

(10991, 10991)

In [218]:
year_state = pivot.index.values

In [219]:
names = pivot.columns

In [222]:
U = pd.DataFrame(U, index=year_state)

In [223]:
VT = pd.DataFrame(VT, columns=names)

In [254]:
# U.iloc[:,:2]
# sigma[:20]
# VT.iloc[:10,:]

In [263]:
nmf = NMF(n_components=10)
W_sklearn = nmf.fit_transform(pivot)
H_sklearn = nmf.components_

In [234]:
def reconst_mse(target, left, right):
    return (array(target - left.dot(right))**2).mean()

def describe_nmf_results(document_term_mat, W, H, n_top_words = 15):
    print("Reconstruction error: %f") %(reconst_mse(document_term_mat, W, H))
    for topic_num, topic in enumerate(H):
        print("Topic %d:" % topic_num)
        print(" ".join([feature_words[i] \
                for i in topic.argsort()[:-n_top_words - 1:-1]]))
    return

In [241]:
# describe_nmf_results(pivot, W_sklearn, H_sklearn)

![image](nmf_image.png)

In [243]:
#I've structured the features and rows differently than we did in the sprint
# The names are the features of my original matrix 
features = pivot.columns

In [245]:
#This matrix is vertically shaped, with latent features as the columns
W_sklearn.shape

(5317, 15)

In [246]:
#This matrix is horizontally shaped, with latent features as the rows
H_sklearn.shape

(15, 10991)

In [290]:
def print_top_features(H, features, top_features):
    for latent_num, latent in enumerate(H):
        print "Latent %d" % latent_num
        print [features[i] for i in latent.argsort()[:-top_features-1:-1]]
        print 

# def print_index_features(W, features, top_features):
#     for latent_num, latent in enumerate(W.T):
#         print "Latent %d" % latent_num
#         print [features[i] for i in latent.argsort()[:-top_features-1:-1]]
#         print 

In [291]:
print_top_features(H_sklearn, features, 10)

Latent 0
[('mary', 'F'), ('deborah', 'F'), ('ronald', 'M'), ('ruth', 'F'), ('betty', 'F'), ('joyce', 'F'), ('jean', 'F'), ('joan', 'F'), ('daniel', 'M'), ('nancy', 'F')]

Latent 1
[('robert', 'M'), ('barbara', 'F'), ('elizabeth', 'F'), ('dorothy', 'F'), ('patricia', 'F'), ('francis', 'M'), ('jane', 'F'), ('matthew', 'M'), ('betty', 'F'), ('mildred', 'F')]

Latent 2
[('john', 'M'), ('sarah', 'F'), ('mark', 'M'), ('jennifer', 'F'), ('daniel', 'M'), ('tina', 'F'), ('crystal', 'F'), ('teresa', 'F'), ('kyle', 'M'), ('frank', 'M')]

Latent 3
[('william', 'M'), ('edward', 'M'), ('anna', 'F'), ('kenneth', 'M'), ('jessica', 'F'), ('hazel', 'F'), ('virginia', 'F'), ('sharon', 'F'), ('timothy', 'M'), ('harold', 'M')]

Latent 4
[('michael', 'M'), ('amy', 'F'), ('joseph', 'M'), ('betty', 'F'), ('jessica', 'F'), ('christina', 'F'), ('timothy', 'M'), ('tyler', 'M'), ('albert', 'M'), ('stephen', 'M')]

Latent 5
[('thomas', 'M'), ('christopher', 'M'), ('joseph', 'M'), ('elizabeth', 'F'), ('matthew', 'M

In [286]:
table_index = pivot.index

In [295]:
name_pivot = pd.pivot_table(sample, values='count_norm',index=sample.name_gender, columns=sample.index)
name_pivot = name_pivot.fillna(0)

In [296]:
nmf = NMF(n_components=10)
W_sklearn = nmf.fit_transform(name_pivot)
H_sklearn = nmf.components_

In [298]:
state_features = name_pivot.columns

In [299]:
print_top_features(H_sklearn,state_features, 10)

Latent 0
[(1942, 'NV'), (1929, 'NH'), (1939, 'NH'), (1946, 'NH'), (1957, 'MA'), (1912, 'NE'), (1946, 'NV'), (1943, 'NV'), (1935, 'AK'), (1998, 'NH')]

Latent 1
[(1926, 'MN'), (1940, 'MO'), (1934, 'TN'), (1929, 'SC'), (1945, 'DE'), (1945, 'WV'), (1928, 'VT'), (1958, 'NE'), (1917, 'WV'), (1925, 'VT')]

Latent 2
[(1916, 'NV'), (1944, 'DE'), (1939, 'RI'), (1911, 'CO'), (1969, 'RI'), (1931, 'NY'), (1920, 'MT'), (1979, 'NH'), (1922, 'MS'), (1972, 'DE')]

Latent 3
[(1926, 'AK'), (1951, 'PA'), (1917, 'OR'), (1922, 'OH'), (1937, 'IN'), (1961, 'DE'), (1925, 'NV'), (1932, 'VA'), (1965, 'MT'), (1910, 'WY')]

Latent 4
[(1945, 'VT'), (1968, 'WY'), (1949, 'DE'), (1980, 'IL'), (1956, 'WA'), (1947, 'IL'), (1988, 'KS'), (1981, 'AZ'), (1941, 'WA'), (1955, 'MD')]

Latent 5
[(1913, 'WY'), (1912, 'WY'), (1934, 'DC'), (1977, 'DE'), (1981, 'AK'), (1969, 'AK'), (1950, 'IL'), (1968, 'NJ'), (1963, 'VT'), (1991, 'IA')]

Latent 6
[(1974, 'NV'), (1969, 'CT'), (1952, 'IL'), (1967, 'AK'), (1956, 'ME'), (1944, 'OH'), 